# Data Pipelines Guided Lesson

Aqui vamos pegar a lista em .csv das açoes que vamos trazer para nossa análise

In [4]:
import pandas as pd

def get_tickers(path):
    companies = pd.read_csv(path)
    ticker_list = list(companies['Ticker'])
    print('Retrieved', str(len(ticker_list)), 'ticker symbols.')
    return ticker_list

### Aquisição dos dados
Essa API permite que solicitamos diretamente no site os dados das 30 empresas que vamos trabalhar

In [5]:
! pip install quandl

In [6]:
import quandl

def get_prices(ticker):
    print('Retrieving data for', ticker)
    quandl.ApiConfig.api_key = ""
    prices = quandl.get('WIKI/' + ticker)['Adj. Close'].reset_index()
    prices['Ticker'] = ticker
    return prices

data = []

ticker_list = get_tickers('companies.csv')

for ticker in ticker_list:
    prices = get_prices(ticker)
    data.append(prices)

Retrieved 30 ticker symbols.
Retrieving data for AAPL
Retrieving data for NFLX
Retrieving data for AMZN
Retrieving data for MSFT
Retrieving data for GOOG
Retrieving data for TSLA


LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

Agora precisamos reajustar o dataframe para ficarmos com as datas nas linhas e as empresas nas colunas

In [ ]:
df = pd.concat(data, sort=True)

In [ ]:
df.head()

O metodo abaixo faz o calculo da variaçao do preço da ação, ou seja, quanto mudou o preço de um dia para o outro

In [ ]:
pivot = df.pivot_table(
    values='Adj. Close', columns='Ticker', index='Date')

In [ ]:
pivot.head()

In [ ]:
returns = pivot.pct_change()

In [ ]:
returns.head()

Agora vamos encontrar a média e o desvio padrão dos preços para cada empresa, considerando todo o histórico que temos disponível.
E se juntássemos as duas quantidades num índice só? Será que obteríamos alguma informação útil? A minha ideia é que a razão média/desvio-padrão pode dar uma noção de quão defensivo um papel é, isto é, quanto maior esta razão, maior o retorno e menor a volatilidade daquele papel. Vamos fazer o cálculo desse índice para algumas ações e ver o que obtemos.

In [ ]:
means = pd.DataFrame(returns.tail(30).mean())
std = pd.DataFrame(returns.tail(30).std())
ratios = pd.concat([means, std], axis=1).reset_index()
ratios.columns = ['Company', 'Mean', 'Std']
ratios['Ratio'] = ratios['Mean']/ratios['Std']


In [ ]:
print(ratios)

In [ ]:
top10 = ratios.sort_values('Ratio', ascending=False).head(10)

In [ ]:
top10.head()

In [ ]:
def corr_matrix(df, days=30):
    corr_matrix = df.tail(days).corr()
    return corr_matrix

target_list = returns[list(top10['Company'])]
correlation = corr_matrix(target_list)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def barchart(df, x, y, length=8, width=14, title=""):
    df = df.sort_values(x, ascending=False)
    plt.figure(figsize=(width,length))
    chart = sns.barplot(data=df, x=x, y=y)
    plt.title(title + "\n", fontsize=16)
    return chart

bar_plot = barchart(top10, 'Ratio', 'Company', title='Stock Return vs. Risk Ratios')

bar_plot

In [ ]:
import numpy as np

def correlation_plot(corr, title=""):
    mask = np.zeros_like(corr, dtype=np.bool)
    mask[np.triu_indices_from(mask)] = True

    plt.subplots(figsize=(15, 10))
    cmap = sns.diverging_palette(6, 255, as_cmap=True)
    
    chart = sns.heatmap(corr, mask=mask, cmap=cmap, center=0, linewidths=.5, annot=True, fmt='.2f')
    plt.title(title, fontsize=16)
    return chart

corr_plot = correlation_plot(correlation, title='Stock Return Correlation')

## Fronteira Eficiente - Como alocar ações num portfólio? 

A grande contribuição que Markowitz trouxe ao mundo dos investimentos foram as suas considerações sobre o efeito da covariância entre as ações individuais e como ela impacta o risco do portfólio final.

 

Podemos ter um portfólio com um risco muito mais baixo se combinarmos ativos que são negativamente correlacionados porque quando um subir o outro cairá e vice-versa, e assim teremos uma volatilidade muito menor. O que procuramos no final é uma correlação negativa no curto prazo e correlação positiva no longo prazo.


Há uma série de algoritmos diferentes para se chegar ao portfólio ótimo para cada combinação de ações. 

 
Suponha que temos um conjunto de ativos e queremos descobrir como eles devem ser alocados analisando seu retorno/risco, entre outras coisas. Vamos supor que para montar nosso portfólio de ações tivéssemos que escolher entre os seguintes papéis:

In [ ]:
pivot2 = pivot[pivot.columns[pivot.columns.isin(['NFLX','GCI','AMZN','WM', 'UA'])]]
#pivot2=pivot2.dropna().head()

In [ ]:
pivot2.head()

In [ ]:
import numpy as np

log_ret=np.log(pivot2/pivot2.shift(1))


In [ ]:
log_ret.head()

In [ ]:
np.random.seed(42)
num_ports = 10000
all_weights = np.zeros((num_ports, len(pivot2.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for x in range(num_ports):
    # pesos
    weights = np.array(np.random.random(5))
    weights = weights/np.sum(weights)

    # salvando os pesos
    all_weights[x,:] = weights
    
    # retorno esperado
    ret_arr[x] = np.sum( (log_ret.mean() * weights ))*252
    
    # risco esperado
    vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov(), weights)))* np.sqrt(252)
    
    # retorno/risco
    sharpe_arr[x] = ret_arr[x]/vol_arr[x]

In [ ]:
max_sr_ret = ret_arr[sharpe_arr.argmax()]
max_sr_vol = vol_arr[sharpe_arr.argmax()]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(max_sr_vol, max_sr_ret,c='red', s=50) # ponto vermelho
plt.show()

Podemos perceber que para cada nível de retorno alvo, há um portfólio ótimo. Logo, vamos supor que temos diferentes níveis de retornos esperados, como mostrados no gráfico. 

Em outras palavras, esses portfólios refletem os pesos de todas essas ações que nos dá o menor risco para cada retorno alvo. Agora, podemos fazer esse cálculo para todos os retornos possíveis, do mínimo até o máximo. Se fizermos isso, vamos obter uma curva. 

Essa curva é chamada de fronteira eficiente na teoria de Markowitz. Ela determina qual o risco que teremos dado um retorno alvo que queremos atingir. O portfólio de máximo retorno nesse caso seria obviamente no ponto vermelho, já que no nosso gráfico é o portifolio de maior retorno. À medida em que reduzimos o retorno alvo, nós encontramos vários outros portfólios com pesos diferentes para cada ação, até atingirmos aquele de menor risco, que é o ponto de inflexão da curva.


Interessante notar que ao reduzirmos nosso retorno alvo, muitas vezes a fronteira eficiente faz uma curva na direção contrária, aumentando o risco. Por isso, na prática, a maioria das pessoas analisa a curva da inflexão para cima, afinal quem quer um portfólio com menor retorno e maior risco, não é mesmo?


Como vocês já podem ter imaginado, não existem portfólios acima da fronteira eficiente, e todos os portfólios que se localizam abaixo dela são sub-ótimos de alguma forma.
